In [5]:
from rdkit import Chem
from itertools import chain
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

def unique_list_reactants (reactants_tuple: tuple) -> float:
    '''
    A tuple of tuples, containing the reactants (as Mol objects) needed to form the target molecule is received.
    This function returns a list of unique lists of the reactants (as Mol objects),
    accounting for the symmetry of the molecule.
    '''
    reactants_list = []
    for r in reactants_tuple:   #The tuple of tuples is transformed into a list of lists
        rxn_site = []           #of reactants (as String objects, using SMILES)
        for molecule in r:
            rxn_site.append(Chem.MolToSmiles(molecule))
        reactants_list.append(rxn_site)
    reactants_set = {tuple(r) for r in reactants_list} #The list is transformed in a set of tuples, to have unique elements
    reactants_list_unique = [list(r) for r in reactants_set] #The set of tuples is trnasformed into a list of unique lists
    reactants_unique_mol = []
    for r in reactants_list_unique: #Here, the list of unique list of reactants (as Mol objects) is created
        mols = []
        for smiles in r:
            mols.append(Chem.MolFromSmiles(smiles))
        reactants_unique_mol.append(mols)
    return reactants_unique_mol
    
def C_S_disconnection (mol: Chem.rdchem.Mol) ->int:
    if not isinstance (mol, Chem.rdchem.Mol):
        raise TypeError(
            f" Invalid type {type(mol)}: 'mol'  " )
    '''
    The function receives a mol object and verrifes if a C(sp3 hybridized)-S bond is present
    If yes, the reactants from which the bond could be formed: 
    an alkyl iodide and a thiol in the presence of potassium carbonate are displayed and 1 is returned
    If the sulfur atom is bonded to two C(sp3 hybridized) atoms, 
    there are two possible disconnections for each S atom.
    If the molecule has a certain symmetry, the number of disconnections is reduced.
    If the bond is not present, the function returns 0.
    
    '''
    
    if mol.HasSubstructMatch(Chem.MolFromSmarts('[^3][S]')): 
        print (f"C(sp3)-S disconection available")
        print (f"--------------------------------------")
        iodine_smiles = 'I'
        potassium_carbonate_smiles ='C(=O)([O-])[O-].[K+].[K+]'
        iodine = Chem.MolFromSmiles(iodine_smiles)
        potassium_carbonate = Chem.MolFromSmiles(potassium_carbonate_smiles)
        rxn = AllChem.ReactionFromSmarts('[C^3:1][S:2].[I:3]>>[C:1][I:3].[S:2]') #Searches for the bond pattern and returns reactants
        reactants = unique_list_reactants(rxn.RunReactants((mol,iodine )))       #to make the bond in a list of lists; each list = reactive site
        opt = 1 #Parameter to count the number of possible reactants that form C(sp3 hybridized)-S bonds
        for r in reactants: #Prints the reactants
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [r[1], potassium_carbonate]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            opt += 1
            print (f"--------------------------------------")
        print(f"--------------------------------------")
        return 1
    return 0
    
def ester_disconnection (mol: Chem.rdchem.Mol) ->int:

    if mol.HasSubstructMatch(Chem.MolFromSmarts('[^2](=[O])[O][C]')): #Verifies if an ester is present 
        print (f"Ester disconnection available")
        print (f"--------------------------------------")
        proton_smiles = '[H+]'
        proton = Chem.MolFromSmiles(proton_smiles)
        hydroxyl_smile = 'O'
        hydroxyl = Chem.MolFromSmiles(hydroxyl_smile)
        rxn = AllChem.ReactionFromSmarts('[C^2:1](=[O:2])[O:3][C:4].[O:5]>>[C^2:1](=[O:2])[O:5].[O:3][C:4]') #Searches for the ester and returns
        reactants = unique_list_reactants(rxn.RunReactants((mol, hydroxyl))) #reactants to make the ester in a list of lists; each list = reactive site
        opt = 1 #Parameter to count the number of possible reactants that form esters
        for r in reactants: #Prints the reactants and the catalyst
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [r[1]]
            catalyst = [proton]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            print(f"Catalyst")
            display(Draw.MolsToGridImage(catalyst))
            opt += 1
            print (f"--------------------------------------")
        print (f"--------------------------------------")
        return 1
    return 0

def acyl_chloride_disconnection (mol: Chem.rdchem.Mol) ->list:

    if mol.HasSubstructMatch(Chem.MolFromSmarts('[C^2](=[O])[Cl]')): #Verifies if the acyl chloride is present
        print (f"Acyl chloride disconnection available")
        print (f"--------------------------------------")
        oxygen_smiles = 'O'
        chlorine_smiles = '[Cl]'
        SOCl2_smiles ='Cl[S](Cl)=O'
        oxygen = Chem.MolFromSmiles(carbonyl_smiles)
        chlorine = Chem.MolFromSmiles(Chloro_smiles)
        SOCl2 = Chem.MolFromSmiles(SOCl2_smiles)
        rxn = AllChem.ReactionFromSmarts('[C^2:1](=[O:2])[Cl:3].[O:4]>>[C^2:1](=[O:2])[O:4].[Cl:3]') #Searches for the acyl chloride and returns reactants
        reactants = unique_list_reactants(rxn.RunReactants((mol,carbonyl)))#to make the acyl chloride in a list of lists; each list = reactive site
        opt = 1 #Parameter to count the number of possible reactants that form acyl chlorides
        for r in reactants: #Prints the reactants
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [SOCl2]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            opt += 1
            print (f"--------------------------------------")
        print (f"--------------------------------------")
        return 1
    return 0

def amine_disconnection (mol: Chem.rdchem.Mol) ->list:
    """
    Checks for the presence of an alkyl amine, subtituted with at least one C(sp3).

    If such a pattern is found:
    Identifies and displays reactants from which the pattern could be formed: a silyl enol ether and a carbonyl compound,
    with pyridine, also displayed, as a catalyst.
    Displays the reaction intermediate.
    Displays the reduction conditions of the intermediate.
    Ensures that, if the molecule has certain symmetry, the number of disconnections is reduced.
    Returns a list containing 1 and a list of the reactants, catalyst, intermediates and hydrogenation conditions.

    If the pattern is not present:
    Returns a list containing 0.

    Parameters:
    mol (Chem.rdchem.Mol): The molecule to be analyzed.

    Returns:
    list: [1, reactants_returned] if the pattern is found, where reactants_returned is a list of reactants.
    [0] if the pattern is not found.

    Raises:
    TypeError: If the input is not a Chem.rdchem.Mol object.
    """
    # Validate input type
    if not isinstance(mol, Chem.rdchem.Mol):
        raise TypeError(
            f"Invalid type :{type(mol)}: 'mol'"
            f"Should be passed as a mol object."
        )

    if mol.HasSubstructMatch(Chem.MolFromSmarts('[C^3][N]')): #Verifies if the alkyl amine (containing at least one sp3 subtituent is present) 
        print (f"Amine disconnection available")
        print (f"--------------------------------------")
        oxygen_smiles = 'O'
        chlorine_smiles='Cl'
        LiAlH4_smiles ='[Li+].[AlH4-]'
        pyridine_smiles = 'c1ccccn1'
        oxygen = Chem.MolFromSmiles(oxygen_smiles)
        chlorine = Chem.MolFromSmiles(chlorine_smiles)
        LiAlH4 = Chem.MolFromSmiles(LiAlH4_smiles)
        pyridine = Chem.MolFromSmiles(pyridine_smiles)
        rxn = AllChem.ReactionFromSmarts('[N:1][C^3:2].[Cl:3].[O:4]>>[N:1].[C^2:2](=[O:4])[Cl:3]') #Searches for the bond pattern and returns reactants
        reactants = unique_list_reactants(rxn.RunReactants((mol,Chloro,carbonyl)))#to make the bond in a list of lists; each list = reactive site
        reactants_returned = [] #List of reactants that are going to be returned 
        opt = 1 #Parameter to count the number of possible reactants that form C(sp3 hybridized)-N bonds
        for r in reactants: #Prints the reactants
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [r[1]]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            print(f"Catalyst")
            display(Draw.MolsToGridImage([pyridine]))
            print (f"Once combined, the reactants give the following intermediate")
            #The reaction intermediate is formed and is displayed afterwards
            rxn_intermediate = AllChem.ReactionFromSmarts('[N^3:1].[Cl:2][C:3]=[O:4]>>[N^3:1][C:3]=[O:4].[Cl:2]')
            intermediate = unique_list_reactants(rxn_intermediate.RunReactants((r[0],r[1])))
            display(Draw.MolsToGridImage([intermediate[0][0]]))
            print (f"This intermediate needs to be reduced" #The reducing conditions of the intermediate are displayed
               f" in order to obtain the target molecule. The following reducing agent could be used:")
            display(Draw.MolsToGridImage([LiAlH4]))
            #The reactants corresponding to each reactive site are added in the list that is returned
            reactants_returned.append(r[0])
            reactants_returned.append(r[1])
            reactants_returned.append(catalyst)
            reactants_returned.append(intermediate[0][0])
            reactants_returned.append(LiAlH4)
            opt += 1
            print (f"--------------------------------------")
        print (f"--------------------------------------")
        return [1, reactants_returned]
     return [0]
        
        

def amino_alcohol_1_2 (mol: Chem.rdchem.Mol) ->list:

    if mol.HasSubstructMatch(Chem.MolFromSmarts('[C^3]([OH])[C^3][NH]')): #Searches for a 1,2 amino-alcohol 
        print (f"1,2 amino-alcohol")
        print (f"--------------------------------------")
        NaNH2_smiles='[NH2-].[Na+]'
        NaNH2 = Chem.MolFromSmiles(NaNH2_smiles)
        rxn = AllChem.ReactionFromSmarts('[C^3:1]([OH:2])[C^3:3][NH:4]>>[C^3:1]1[O:2][C^3:3]1.[N:4]') #The pattern is searched and reactants
        reactants = unique_list_reactants(rxn.RunReactants((mol,NH3)))#that could make it are returned as a list of lists; each list = reactive site
        reactants_returned = [] #List of reactants that are going to be returned
        opt = 1 #Parameter to count the number of possible reactants that form the 1,2 amino-alcohol
        for r in reactants: #Reactants are displayed
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [r[1], NaNH2]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            opt += 1
            #The reactants corresponding to each reactive site are added in the list that is returned
            reactants_returned.append(r[0])
            reactants_returned.append(r[1])
            reactants_returned.append(NaNH2)
            print (f"--------------------------------------")
        print (f"--------------------------------------")
        return [1, reactants_returned]
    return [0]

def grignard (mol: Chem.rdchem.Mol) ->list:
    
    if mol.HasSubstructMatch(Chem.MolFromSmarts('[C][C^3][OH]')): #Verifies if an alcohol is present
        print (f"Grignard disconnection available")
        print (f"--------------------------------------")
        # Define SMILES for necessary compounds
        Mg_smiles='[Mg]'
        Br_smiles='Br'
        O_smiles='O'
        THF_smiles='C1CCOC1'
        Mg = Chem.MolFromSmiles(Mg_smiles)
        Br = Chem.MolFromSmiles(Br_smiles)
        O = Chem.MolFromSmiles(O_smiles)
        THF = Chem.MolFromSmiles(THF_smiles)
        rxn = AllChem.ReactionFromSmarts('[C:1][C^3:2][O:3].[Mg:4].[Br:5]>>[C^2:2](=[O:3]).[C:1][Mg:4][Br:5]') #Searches for the alcohol and returns
        reactants = unique_list_reactants(rxn.RunReactants((mol,Mg,Br))) #the reactants to make it in a list of lists; each list = reactive site
        reactants_returned = [] #List of reactants that are going to be returned
        opt = 1 #Parameter to count the number of possible reactants that form C(sp3 hybridized)-O bonds
        for r in reactants: #Reactants and solvent are displayed
            print(f"Option {opt}")
            reactant_1 = [r[0]]
            reactant_2 = [r[1]]
            solvent = [THF]
            print(f"Reactant 1")
            display(Draw.MolsToGridImage(reactant_1))
            print(f"Reactant 2")
            display(Draw.MolsToGridImage(reactant_2))
            print(f"Solvent")
            display(Draw.MolsToGridImage(solvent))
            opt += 1
            #The reactants corresponding to each reactive site are added in the list that is returned
            reactants_returned.append(r[0])
            reactants_returned.append(r[1])
            reactants_returned.append(THF)
            print (f"--------------------------------------")
        print(f"--------------------------------------")
        return [1, reactants_returned]
    return [0]


def disconnections (mol_smiles: str):
    '''
    This function takes a SMILE of a molecule and
    displays all the known disconnections.
    '''
    mol = Chem.MolFromSmiles(mol_smiles) #Transform SMILE in mol object
    print (f"The molecule inserted looks like this:")
    display(mol)
    print (f"--------------------------------------")
    print (f"--------------------------------------")
    sum = 0 #Parameter that keeps track of the number of the types of disconnections found in the molecule
    known_disc = [C_S_disconnection, ester_disconnection, acyl_chloride_disconnection, amine_disconnection, epoxyde_disconnection, grignard]
    for disc in known_disc:
        sum += disc(mol)
    if not sum: #In case no disconnection was found
        print(f"The molecule contains no known disconnections")




def main ():
    mol_smiles = 'CC(O)CNC'
    disconnections (mol_smiles)

if __name__ == '__main__':
    main()




IndentationError: unindent does not match any outer indentation level (<tokenize>, line 201)

In [18]:
import requests

### molecule name --> smiles configuration
def smiles(molecule_name):
    url = f"https://cactus.nci.nih.gov/chemical/structure/{molecule_name}/smiles"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text.strip()
    else:
        return None

smiles('propanoyl chloride')

'CCC(Cl)=O'